SIDE NOTE: would love to get any critiques, hints, tips, oberservations on any level not just on ML (but obviously ML as the priority).  Beginner Python learner (though I took the Full Stack Course) so any suggestions there are welcome.  Thanks.

Will need to import these libs plus setting some global variables

In [9]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
import matplotlib.cm as cm
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Might need to change the path of the included libraries.
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches

# Variables
round_number = 27 # for MLS only
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled', 'games_played', 'round']
sub_cols = ['current_formation', 'avg_goals_against', 'goal_diff', 'win_percentage', 'sos',
           'opp_win_percentage', 'opp_sos', 'current_team_yellow_cards', 'current_team_corner_kicks', 'current_team_first_half_goals', 'current_team_sec_half_goals', 
           'opp_team_yellow_cards', 'opp_team_corner_kicks', 'opp_team_first_half_goals', 'opp_team_sec_half_goals']

all_models = ['log', 'svc', 'gmm', 'knn', 'gnb']

""" this variable 'testing' should be false if using CSV's and not pulling from the database. """
testing = False

Pulling in the data either from the Database or the CSV (CSV for testers).  Data has some added features in between the database and 'raw_data'.  Essentially trying 3 different versions of the data in this set.  One version contains all the features.  Another version has the standard features plus the calculated features of the home/away stats for a matchup.  The last version has the standard features plus the 'extended features' which not only includes the stats of the two teams in the match up but also the current teams previous opponents and the current teams opponents of the team opponents in that matchup.

In [2]:
data_csv = 'round' + str(round_number) + '.csv'

if testing:
    raw_data = form_data.run_data()
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)

print('Data Loaded...')

Data Loaded...


## Training

Functions that trains basic models (and saves them if need be).  'load_models' will  load exisiting models to save time in testing.

In [14]:
# Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

"""No Cross-Validation and No Tuning"""
def train_models(round_num, X, y, models):
    
    if os.path.isdir("/models/" + str(round_num)):
        print('Making New Directory for the Round')
        os.chdir('/Users/senzari/Machine_Learning/stats/src/models')
        os.makedirs(str(round_num))
        os.chdir('/Users/senzari/Machine_Learning/stats/src')
        
    finished_models = []
    
    for i in models:
        
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        
        
        if i == 'log':
            log = form_model.build_model(X, y, i)
            joblib.dump(log, model_round)
            finished_models.append(log)
        elif i == 'svc':
            svc = form_model.build_model(X, y, i)
            joblib.dump(svc, model_round)
            finished_models.append(svc)
        elif i == 'gmm':
            gmm = form_model.build_model(X, y, i)
            joblib.dump(gmm, model_round)
            finished_models.append(gmm)
        elif i == 'knn':
            kmeans = form_model.build_model(X, y, i)
            joblib.dump(kmeans, model_round)
            finished_models.append(kmeans)
        elif i == 'gnb':
            gnb = form_model.build_model(X, y, i)
            joblib.dump(gnb, model_round)
            finished_models.append(gnb)
    
    return finished_models
        
        
def load_models(round_num):
    
    loaded_models = []
    
    for i in all_models:
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        if i == 'log':
             log = joblib.load(model_round)
             loaded_models.append(log)
        if i == 'svc':
             svc= joblib.load(model_round)
             loaded_models.append(svc)
        elif i == 'gmm':
            gmm = joblib.load(model_round)
            loaded_models.append(gmm)
        elif i == 'knn':
            kmeans = joblib.load(model_round)
            loaded_models.append(kmeans)
        elif i == 'gnb':
            gnb = joblib.load(model_round)
            loaded_models.append(gnb)
        
        print("Success :: Loaded - " + str(i))
        
    return loaded_models

# raw_X, raw_y = split_target(raw_data)

pd.set_option('display.max_columns', 500)

""" Need to format data to convert goals scored to the correct category"""
#formatted_data = raw_data.drop('points', 1)
formatted_data = raw_data.copy()
def set_group(goals):
    if goals >= 2:
        return 1
    elif goals < 2:
        return 0
    
formatted_data['converted_goals'] = formatted_data.apply(lambda row: set_group(row['goals']), axis=1)

#formatted_data = formatted_data.drop(sub_cols + ignore_cols + ['goals'], 1)

def set_rpi_quartile(round_number, data, isCur):
    power_rankings = pd.DataFrame()
    power_list = []
    
    if isCur:
        rpi = 'rpi'
        team_id = 'team_id'
    else:
        rpi = 'opp_rpi'
        team_id = 'opp_id'
        
    td = data.loc[(data["round"] == round_number)]
    
    if not td.empty:
        s = td.loc[:, [team_id, rpi]]
        power_rankings = power_rankings.append(s, ignore_index=False)
        power_rankings = power_rankings.sort_values(rpi, ascending=False)
        for i, power in power_rankings.iterrows():
            power_list.append(power)

        pr = np.array(power_rankings.loc[:, rpi])
        qqs = np.percentile(pr, [25, 50, 75, 100])
        quartiles = [0, .3333, .6666, 1]
        idx = len(pr)
        for i in range(len(qqs)):
            a = np.where( pr[0:idx] <= qqs[i] )
            pr[a] = quartiles[i]
            idx = a[0][0]
            
        return pr, power_rankings.index
    else:
        return None, None
    
mls_data = formatted_data.copy()
mls_data = mls_data.loc[mls_data["team_id"] < 41]
mls_data['rpi_quartiled'] = pd.Series(None, index=mls_data.index) 
mls_min = mls_data['round'].min()
mls_max = mls_data['round'].max() + 1

epl_data = formatted_data.copy()
epl_data = epl_data.loc[epl_data["team_id"] > 40]
epl_min = epl_data['round'].min()
epl_max = epl_data['round'].max() + 1

for x in range(mls_min, mls_max):
    
    power_rankings, idx = set_rpi_quartile(x, mls_data, True)
    opp_power_rankings, opp_idx = set_rpi_quartile(x, mls_data, False)

    if idx is not None:
        mls_data.loc[idx, "rpi_quartiled"] = power_rankings
        
    if opp_idx is not None:
        mls_data.loc[opp_idx, "opp_rpi_quartiled"] = opp_power_rankings
        
for x in range(epl_min, epl_max):
    
    power_rankings, idx = set_rpi_quartile(x, epl_data, True)
    opp_power_rankings, opp_idx = set_rpi_quartile(x, epl_data, False)

    if idx is not None:
        epl_data.loc[idx, "rpi_quartiled"] = power_rankings
        
    if opp_idx is not None:
        epl_data.loc[opp_idx, "opp_rpi_quartiled"] = opp_power_rankings
        
all_data = mls_data
all_data = all_data.append(epl_data)
#display(all_data.head())
goal_data = all_data.copy()
points_data = all_data.copy()
# Drop the original RPI features and everything else
goal_data = goal_data.drop(sub_cols + ignore_cols + ['goals'] + ['rpi', 'opp_rpi'], 1)
display(goal_data.head())
#all_data.to_csv('all.csv')

(goal_y, goal_X) = model_libs._extract_target(goal_data, 'converted_goals')

,is_home,avg_points,avg_goals_for,margin,goal_efficiency,opp_avg_points,opp_avg_goals,opp_margin,opp_goal_efficiency,goals_op_ratio,ball_safe_op_ratio,goal_attempts_op_ratio,current_team_possession,current_team_attacks,current_team_dangerous_attacks,current_team_shots_on_target,current_team_shots_total,current_team_ball_safe,current_team_goal_attempts,current_team_saves,current_team_goal_kicks,opp_team_possession,opp_team_attacks,opp_team_dangerous_attacks,opp_team_shots_on_target,opp_team_shots_total,opp_team_ball_safe,opp_team_goal_attempts,opp_team_saves,opp_team_goal_kicks,points,converted_goals,rpi_quartiled,opp_rpi_quartiled
0,0,2.000000,1.333333,-0.333333,0.117647,1.000000,2.000000,0.666667,0.000000,0.666667,0.759644,1.346154,50.333333,103.333333,60.666667,6.666667,11.666667,85.333333,11.666667,1.666667,5.666667,48.666667,113.666667,45.333333,5.333333,8.666667,112.333333,8.666667,4.333333,8.666667,3,1,1.0000,0.0000
1,1,2.000000,1.666667,0.000000,0.333333,0.333333,0.333333,-1.000000,0.000000,5.000000,0.871166,1.391304,50.666667,112.000000,56.333333,5.666667,10.666667,94.666667,10.666667,1.666667,5.333333,47.666667,110.666667,60.000000,2.333333,7.666667,108.666667,7.666667,1.666667,9.000000,1,0,0.3333,0.0000
2,1,2.333333,2.000000,1.666667,0.052632,0.333333,0.666667,-0.333333,0.166667,3.000000,1.025090,1.406250,49.333333,113.000000,56.000000,6.333333,15.000000,95.333333,15.000000,3.000000,5.666667,54.000000,97.666667,57.000000,6.333333,10.666667,93.000000,10.666667,2.666667,9.000000,1,1,0.6666,0.0000
3,0,1.666667,2.000000,1.000000,0.153846,1.000000,1.333333,-0.666667,0.285714,1.500000,0.957447,2.157895,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,44.666667,90.666667,44.666667,3.666667,6.333333,94.000000,6.333333,2.666667,9.666667,3,1,0.6666,1.0000
4,1,1.666667,2.000000,1.000000,0.153846,0.000000,1.333333,-0.666667,0.285714,1.500000,0.957447,2.157895,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,44.666667,90.666667,44.666667,3.666667,6.333333,94.000000,6.333333,2.666667,9.666667,3,1,0.6666,0.6666


### RUNNING CLASSIFICATION MODEL ON IF TEAMS SCORE 0-1 OR 1-2 ON GAMES

In [19]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    
    display(new_data.head())
    
    (y, X) = model_libs._extract_target(new_data, target)
    
    models = train_models(round_number, X, y, models)
    
    return models


#### Running ALL Features 
#models_test_1 = run_features(all_data, [], 'converted_goals', all_models)

models_test_1 = run_features(goal_data, [], 'converted_goals', ['knn'])
(goal_y, goal_X) = model_libs._extract_target(goal_data, 'converted_goals')
knn = models_test_1[0]

actual_y = pd.DataFrame(goal_y.values, columns=['actual'])
predictions = pd.DataFrame(knn.predict(goal_X), columns=['predictions'])
preds = pd.concat([predictions, actual_y], axis=1)
preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
accuracy = np.divide(preds['diff'].sum(),float(len(preds['diff'])))
#display(preds.head())
#print(accuracy)

combined_data = points_data.copy()

combined_data["goal_predictions"] = predictions


#models_test_1 = run_features(combined_data, [], 'points', all_models)

,is_home,avg_points,avg_goals_for,margin,goal_efficiency,opp_avg_points,opp_avg_goals,opp_margin,opp_goal_efficiency,goals_op_ratio,ball_safe_op_ratio,goal_attempts_op_ratio,current_team_possession,current_team_attacks,current_team_dangerous_attacks,current_team_shots_on_target,current_team_shots_total,current_team_ball_safe,current_team_goal_attempts,current_team_saves,current_team_goal_kicks,opp_team_possession,opp_team_attacks,opp_team_dangerous_attacks,opp_team_shots_on_target,opp_team_shots_total,opp_team_ball_safe,opp_team_goal_attempts,opp_team_saves,opp_team_goal_kicks,points,converted_goals,rpi_quartiled,opp_rpi_quartiled
0,0,2.000000,1.333333,-0.333333,0.117647,1.000000,2.000000,0.666667,0.000000,0.666667,0.759644,1.346154,50.333333,103.333333,60.666667,6.666667,11.666667,85.333333,11.666667,1.666667,5.666667,48.666667,113.666667,45.333333,5.333333,8.666667,112.333333,8.666667,4.333333,8.666667,3,1,1.0000,0.0000
1,1,2.000000,1.666667,0.000000,0.333333,0.333333,0.333333,-1.000000,0.000000,5.000000,0.871166,1.391304,50.666667,112.000000,56.333333,5.666667,10.666667,94.666667,10.666667,1.666667,5.333333,47.666667,110.666667,60.000000,2.333333,7.666667,108.666667,7.666667,1.666667,9.000000,1,0,0.3333,0.0000
2,1,2.333333,2.000000,1.666667,0.052632,0.333333,0.666667,-0.333333,0.166667,3.000000,1.025090,1.406250,49.333333,113.000000,56.000000,6.333333,15.000000,95.333333,15.000000,3.000000,5.666667,54.000000,97.666667,57.000000,6.333333,10.666667,93.000000,10.666667,2.666667,9.000000,1,1,0.6666,0.0000
3,0,1.666667,2.000000,1.000000,0.153846,1.000000,1.333333,-0.666667,0.285714,1.500000,0.957447,2.157895,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,44.666667,90.666667,44.666667,3.666667,6.333333,94.000000,6.333333,2.666667,9.666667,3,1,0.6666,1.0000
4,1,1.666667,2.000000,1.000000,0.153846,0.000000,1.333333,-0.666667,0.285714,1.500000,0.957447,2.157895,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,44.666667,90.666667,44.666667,3.666667,6.333333,94.000000,6.333333,2.666667,9.666667,3,1,0.6666,0.6666


-----------------------------------
Training K Neighbors Classifier Model
KNN Score :: 0.786290322581
Finished K-Means Modeling


In [21]:
""" Using converted goals to assist with overall match points"""
combined_data = combined_data.drop(sub_cols + ignore_cols + ['goals'] + ['rpi', 'opp_rpi'], 1)
display(combined_data.head())

,is_home,avg_points,avg_goals_for,margin,goal_efficiency,opp_avg_points,opp_avg_goals,opp_margin,opp_goal_efficiency,goals_op_ratio,ball_safe_op_ratio,goal_attempts_op_ratio,current_team_possession,current_team_attacks,current_team_dangerous_attacks,current_team_shots_on_target,current_team_shots_total,current_team_ball_safe,current_team_goal_attempts,current_team_saves,current_team_goal_kicks,opp_team_possession,opp_team_attacks,opp_team_dangerous_attacks,opp_team_shots_on_target,opp_team_shots_total,opp_team_ball_safe,opp_team_goal_attempts,opp_team_saves,opp_team_goal_kicks,points,converted_goals,rpi_quartiled,opp_rpi_quartiled,goal_predictions
0,0,2.000000,1.333333,-0.333333,0.117647,1.000000,2.000000,0.666667,0.000000,0.666667,0.759644,1.346154,50.333333,103.333333,60.666667,6.666667,11.666667,85.333333,11.666667,1.666667,5.666667,48.666667,113.666667,45.333333,5.333333,8.666667,112.333333,8.666667,4.333333,8.666667,3,1,1.0000,0.0000,1
1,1,2.000000,1.666667,0.000000,0.333333,0.333333,0.333333,-1.000000,0.000000,5.000000,0.871166,1.391304,50.666667,112.000000,56.333333,5.666667,10.666667,94.666667,10.666667,1.666667,5.333333,47.666667,110.666667,60.000000,2.333333,7.666667,108.666667,7.666667,1.666667,9.000000,1,0,0.3333,0.0000,0
2,1,2.333333,2.000000,1.666667,0.052632,0.333333,0.666667,-0.333333,0.166667,3.000000,1.025090,1.406250,49.333333,113.000000,56.000000,6.333333,15.000000,95.333333,15.000000,3.000000,5.666667,54.000000,97.666667,57.000000,6.333333,10.666667,93.000000,10.666667,2.666667,9.000000,1,1,0.6666,0.0000,1
3,0,1.666667,2.000000,1.000000,0.153846,1.000000,1.333333,-0.666667,0.285714,1.500000,0.957447,2.157895,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,44.666667,90.666667,44.666667,3.666667,6.333333,94.000000,6.333333,2.666667,9.666667,3,1,0.6666,1.0000,1
4,1,1.666667,2.000000,1.000000,0.153846,0.000000,1.333333,-0.666667,0.285714,1.500000,0.957447,2.157895,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,44.666667,90.666667,44.666667,3.666667,6.333333,94.000000,6.333333,2.666667,9.666667,3,1,0.6666,0.6666,1


In [23]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    
    display(new_data.head())
    
    (y, X) = model_libs._extract_target(new_data, target)
    
    models = train_models(round_number, X, y, models)
    
    return models


#### Running ALL Features 
models_test_1 = run_features(combined_data, [], 'points', all_models)

,is_home,avg_points,avg_goals_for,margin,goal_efficiency,opp_avg_points,opp_avg_goals,opp_margin,opp_goal_efficiency,goals_op_ratio,ball_safe_op_ratio,goal_attempts_op_ratio,current_team_possession,current_team_attacks,current_team_dangerous_attacks,current_team_shots_on_target,current_team_shots_total,current_team_ball_safe,current_team_goal_attempts,current_team_saves,current_team_goal_kicks,opp_team_possession,opp_team_attacks,opp_team_dangerous_attacks,opp_team_shots_on_target,opp_team_shots_total,opp_team_ball_safe,opp_team_goal_attempts,opp_team_saves,opp_team_goal_kicks,points,converted_goals,rpi_quartiled,opp_rpi_quartiled,goal_predictions
0,0,2.000000,1.333333,-0.333333,0.117647,1.000000,2.000000,0.666667,0.000000,0.666667,0.759644,1.346154,50.333333,103.333333,60.666667,6.666667,11.666667,85.333333,11.666667,1.666667,5.666667,48.666667,113.666667,45.333333,5.333333,8.666667,112.333333,8.666667,4.333333,8.666667,3,1,1.0000,0.0000,1
1,1,2.000000,1.666667,0.000000,0.333333,0.333333,0.333333,-1.000000,0.000000,5.000000,0.871166,1.391304,50.666667,112.000000,56.333333,5.666667,10.666667,94.666667,10.666667,1.666667,5.333333,47.666667,110.666667,60.000000,2.333333,7.666667,108.666667,7.666667,1.666667,9.000000,1,0,0.3333,0.0000,0
2,1,2.333333,2.000000,1.666667,0.052632,0.333333,0.666667,-0.333333,0.166667,3.000000,1.025090,1.406250,49.333333,113.000000,56.000000,6.333333,15.000000,95.333333,15.000000,3.000000,5.666667,54.000000,97.666667,57.000000,6.333333,10.666667,93.000000,10.666667,2.666667,9.000000,1,1,0.6666,0.0000,1
3,0,1.666667,2.000000,1.000000,0.153846,1.000000,1.333333,-0.666667,0.285714,1.500000,0.957447,2.157895,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,44.666667,90.666667,44.666667,3.666667,6.333333,94.000000,6.333333,2.666667,9.666667,3,1,0.6666,1.0000,1
4,1,1.666667,2.000000,1.000000,0.153846,0.000000,1.333333,-0.666667,0.285714,1.500000,0.957447,2.157895,51.333333,103.333333,48.333333,5.666667,13.666667,90.000000,13.666667,2.666667,5.333333,44.666667,90.666667,44.666667,3.666667,6.333333,94.000000,6.333333,2.666667,9.666667,3,1,0.6666,0.6666,1


Training LOG REG Model
Score on Training Set :: 0.621212121212
Score on Test Set :: 0.56
Finished LOG REG Modeling
-----------------------------------
Training SVC Model
F1 score for training set: 0.636320026801
F1 score for test set: 0.483908433554
Finished SVC Modeling
-----------------------------------
Training GMM Modeling
Silhouette Score :: 0.176531929223 for 2 Clusters
Silhouette Score :: 0.0777570041539 for 3 Clusters
Finished GMM Modeling
-----------------------------------
Training K Neighbors Classifier Model
KNN Score :: 0.713709677419
Finished K-Means Modeling
-----------------------------------
Training Gaussian NB Model
F1 score for training set: 0.88377242789
F1 score for test set: 0.840760546643
Finished Gaussian NB Modeling
